In [1]:
library(devtools)
library("fgsea")
library("here")
library(devtools)
library(tidyverse)
library(Matrix)
library(magrittr) # needs to be run every time you start R and want to use %>%
library(dplyr)  

Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis

here() starts at /home/jpm73279/Mendieta_et_al_comparative_single_cell

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘

# Running On Zea Mays

In [21]:
convertTerms <- function(x){
    out <- lapply(seq(1:nrow(x)), function(z){
        gids <- strsplit(as.character(x$V2[z]), "\\,")
        gids <- do.call(c, gids)
        return(gids)
    })
    names(out) <- as.character(x$V1)
    return(out)
}

zm_func_genes <- read.table("/home/jpm73279/genome_downloads/Zm-B73-REFERENCE-NAM-5.0/biological_process.gmt.txt")
zm_functional_annotation <- convertTerms(zm_func_genes)



In [25]:
str(zm_functional_annotation)

List of 5994
 $ mitochondrial_genome_maintenance                                                                                                                             : chr [1:8] "Zm00001eb085130" "Zm00001eb339190" "Zm00001eb414740" "Zm00001eb421290" ...
 $ reproduction                                                                                                                                                 : chr [1:2879] "Zm00001eb065570" "Zm00001eb065660" "Zm00001eb065740" "Zm00001eb065760" ...
 $ vacuole_inheritance                                                                                                                                          : chr [1:2] "Zm00001eb366710" "Zm00001eb146760"
 $ single_strand_break_repair                                                                                                                                   : chr [1:3] "Zm00001eb071490" "Zm00001eb323120" "Zm00001eb045200"
 $ regulation_of_DNA_recombination                    

In [20]:
## BS deseq2 results
zm_bs_denovo <- read.table("/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/Zm.de_novo_genes/Zm.V3.final_annotation.bundle_sheath_ncell_2870_deseq_2_results.tsv",
                          header = TRUE)
    

In [9]:
read_table_run_FGSEA <- function(de_seq2_input, location, functional_annotations){
    
    input_file_name <- here(location, de_seq2_input)
    zm_bs_denovo <- read.table(input_file_name,
                          header = TRUE)
    
    print(head(zm_bs_denovo))
    zm_bs_denovo.selected <- zm_bs_denovo  %>% 
        dplyr::select(gene_name, stat)

    ranks <- deframe(zm_bs_denovo.selected)
    
    fgseaRes <- fgsea(pathways=zm_functional_annotation, stats=ranks, 
                  nperm=10000,
                 minSize = 10, maxSize = 600)
    
    fgseaResTidy <- fgseaRes %>%
        as_tibble() %>%
        arrange(NES)
        
    
    fgseaResTidy$celltype <- de_seq2_input

    return(fgseaResTidy)
}

In [11]:
working_location <- "/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/02.de_novo_markers/Zm/"
deseq2_files <- c("Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.companion_cells.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.epidermis.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.mesophyll.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.mesophyll_developing.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.phloem_sieve_elements.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.procambial_meristem.upregulated_genes.deseq2_output.tsv",
  "Zm.de_novo.protoderm.upregulated_genes.deseq2_output.tsv"
)
all_gathered_cell_type_FGSEA <- lapply(deseq2_files, read_table_run_FGSEA, location = working_location, functional_annotations = zm_functional_annotation)

        gene_name baseMean log2FoldChange      lfcSE     stat        pvalue
1 Zm00001eb121470 391.7874       2.749280 0.10509145 26.16084 7.419693e-151
2 Zm00001eb279660 244.4615       3.142364 0.13461125 23.34399 1.586078e-120
3 Zm00001eb108620 457.1313       1.824858 0.08081114 22.58177 6.547762e-113
4 Zm00001eb144010 284.7238       2.384114 0.10577520 22.53945 1.704387e-112
5 Zm00001eb055880 537.6557       1.771079 0.08023855 22.07267 5.787367e-108
6 Zm00001eb140540 243.9550       2.685079 0.12241730 21.93382 1.236112e-106
           padj
1 1.074705e-146
2 1.531570e-116
3 4.742053e-109
4 9.874877e-109
5 2.395060e-104
6 4.476118e-103


Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.48% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”


        gene_name  baseMean log2FoldChange     lfcSE     stat       pvalue
1 Zm00001eb067750 211.84880       2.047008 0.1269279 16.12733 1.639591e-58
2 Zm00001eb293110  75.47942       3.593037 0.2517598 14.27169 3.285284e-46
3 Zm00001eb147960  69.77435       4.787828 0.3700256 12.93918 2.705034e-38
4 Zm00001eb158370  79.79995       2.877041 0.2252376 12.77336 2.309612e-37
5 Zm00001eb128890  76.22223       3.104283 0.2451075 12.66499 9.243206e-37
6 Zm00001eb077010  67.80537       3.128083 0.2520017 12.41294 2.223227e-35
          padj
1 1.517770e-54
2 2.027458e-42
3 1.252025e-34
4 8.552033e-34
5 2.852145e-33
6 5.880117e-32


Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”


        gene_name baseMean log2FoldChange      lfcSE     stat        pvalue
1 Zm00001eb009780 361.1250       2.481627 0.09494073 26.13870 1.324968e-150
2 Zm00001eb338800 461.5121       1.871236 0.07883866 23.73500 1.569638e-124
3 Zm00001eb338810 210.0486       2.408005 0.12336924 19.51869  7.617336e-85
4 Zm00001eb313510 423.3847       1.649776 0.09047339 18.23493  2.726107e-74
5 Zm00001eb048950 141.1213       2.793619 0.15803663 17.67703  6.302878e-70
6 Zm00001eb413470 232.3360       1.905933 0.11168740 17.06489  2.708995e-65
           padj
1 1.947306e-146
2 1.537932e-120
3  4.478079e-81
4  1.335520e-70
5  2.646669e-66
6  8.847577e-62


...

Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.3% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”


        gene_name baseMean log2FoldChange     lfcSE     stat       pvalue
1 Zm00001eb431760 46.48665       1.840509 0.2507557 7.339852 2.138304e-13
2 Zm00001eb069140 37.03206       1.991020 0.2885927 6.899066 5.234572e-12
3 Zm00001eb060520 30.63472       1.963687 0.3160084 6.214035 5.164109e-10
4 Zm00001eb077010 23.98428       2.130410 0.3558286 5.987181 2.135090e-09
5 Zm00001eb247590 18.10752       2.623274 0.4375132 5.995874 2.023934e-09
6 Zm00001eb408920  8.55272       6.612953 1.1123886 5.944823 2.767558e-09
          padj
1 7.813605e-10
2 1.002857e-08
3 8.480205e-07
4 2.454286e-06
5 2.454286e-06
6 2.892098e-06


Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”


        gene_name baseMean log2FoldChange      lfcSE     stat       pvalue
1 Zm00001eb219710 210.6752       2.584590 0.13411457 19.27151 9.318354e-83
2 Zm00001eb130080 121.1564       3.100511 0.19820892 15.64264 3.729408e-55
3 Zm00001eb048150 115.6033       2.339422 0.16904707 13.83888 1.485070e-43
4 Zm00001eb232100 132.0976       2.150889 0.15645182 13.74793 5.240554e-43
5 Zm00001eb084940 160.4793       2.002388 0.14709986 13.61244 3.377810e-42
6 Zm00001eb053220 400.9869       1.176494 0.09229099 12.74766 3.212641e-37
          padj
1 1.445137e-78
2 2.313501e-51
3 6.580344e-40
4 2.031828e-39
5 1.164106e-38
6 9.058771e-34


Warning message in fgsea(pathways = zm_functional_annotation, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.46% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”


In [12]:
zm_cell_type_FGSEA <- bind_rows(all_gathered_cell_type_FGSEA)

In [13]:
head(zm_functional_annotation)

$mitochondrial_genome_maintenance
[1] "Zm00001eb085130" "Zm00001eb339190" "Zm00001eb414740" "Zm00001eb421290"
[5] "Zm00001eb426210" "Zm00001eb431790" "Zm00001eb294240" "Zm00001eb141480"

$reproduction
   [1] "Zm00001eb065570" "Zm00001eb065660" "Zm00001eb065740" "Zm00001eb065760"
   [5] "Zm00001eb065940" "Zm00001eb065970" "Zm00001eb066340" "Zm00001eb066640"
   [9] "Zm00001eb066780" "Zm00001eb066790" "Zm00001eb066830" "Zm00001eb067080"
  [13] "Zm00001eb067210" "Zm00001eb067270" "Zm00001eb067440" "Zm00001eb067510"
  [17] "Zm00001eb067800" "Zm00001eb068020" "Zm00001eb068390" "Zm00001eb068440"
  [21] "Zm00001eb068550" "Zm00001eb068720" "Zm00001eb068710" "Zm00001eb068860"
  [25] "Zm00001eb068880" "Zm00001eb068930" "Zm00001eb069270" "Zm00001eb069290"
  [29] "Zm00001eb069410" "Zm00001eb069440" "Zm00001eb069480" "Zm00001eb069490"
  [33] "Zm00001eb069500" "Zm00001eb069530" "Zm00001eb069550" "Zm00001eb069670"
  [37] "Zm00001eb069690" "Zm00001eb070050" "Zm00001eb070270" "Zm00001eb070280"
  [41] "Zm00001eb070450" "Zm00001eb070760" "Zm00001eb070860" "Zm00001eb070960"
  [45] "Zm00001eb070980" "Zm00001eb071030" "Zm00001eb071550" "Zm00001eb071970"
  [49] "Zm00001eb071980" "Zm00001eb071990" "Zm00001eb072270" "Zm00001eb072410"
  [53] "Zm00001eb072420" "Zm00001eb072800" "Zm00001eb072940" "Zm00001eb073010"
  [57] "Zm00001eb073110" "Zm00001eb073240" "Zm00001eb073290" "Zm00001eb073370"
  [61] "Zm00001eb073540" "Zm00001eb073810" "Zm00001eb074060" "Zm00001eb074140"
  [65] "Zm00001eb074180" "Zm00001eb074570" "Zm00001eb074930" "Zm00001eb074940"
  [69] "Zm00001eb074990" "Zm00001eb075050" "Zm00001eb075140" "Zm00001eb075200"
  [73] "Zm00001eb075250" "Zm00001eb075420" "Zm00001eb075520" "Zm00001eb075640"
  [77] "Zm00001eb075800" "Zm00001eb075810" "Zm00001eb076080" "Zm00001eb076170"
  [81] "Zm00001eb076180" "Zm00001eb076210" "Zm00001eb076420" "Zm00001eb076460"
  [85] "Zm00001eb076670" "Zm00001eb076890" "Zm00001eb077250" "Zm00001eb077610"
  [89] "Zm00001eb077770" "Zm00001eb077840" "Zm00001eb077960" "Zm00001eb077980"
  [93] "Zm00001eb078280" "Zm00001eb078350" "Zm00001eb078540" "Zm00001eb078720"
  [97] "Zm00001eb078730" "Zm00001eb078790" "Zm00001eb078880" "Zm00001eb078900"
 [101] "Zm00001eb078980" "Zm00001eb079000" "Zm00001eb079310" "Zm00001eb079490"
 [105] "Zm00001eb079500" "Zm00001eb079670" "Zm00001eb079950" "Zm00001eb080690"
 [109] "Zm00001eb081350" "Zm00001eb081420" "Zm00001eb081530" "Zm00001eb081610"
 [113] "Zm00001eb081630" "Zm00001eb081720" "Zm00001eb082060" "Zm00001eb082160"
 [117] "Zm00001eb082270" "Zm00001eb082590" "Zm00001eb082700" "Zm00001eb082750"
 [121] "Zm00001eb082810" "Zm00001eb082920" "Zm00001eb083200" "Zm00001eb083290"
 [125] "Zm00001eb083630" "Zm00001eb083900" "Zm00001eb084030" "Zm00001eb084190"
 [129] "Zm00001eb084710" "Zm00001eb084990" "Zm00001eb085030" "Zm00001eb085190"
 [133] "Zm00001eb085270" "Zm00001eb085290" "Zm00001eb085530" "Zm00001eb086560"
 [137] "Zm00001eb086710" "Zm00001eb087050" "Zm00001eb087250" "Zm00001eb087500"
 [141] "Zm00001eb088350" "Zm00001eb088500" "Zm00001eb088510" "Zm00001eb088790"
 [145] "Zm00001eb088950" "Zm00001eb089080" "Zm00001eb089580" "Zm00001eb089960"
 [149] "Zm00001eb090320" "Zm00001eb090350" "Zm00001eb090620" "Zm00001eb090890"
 [153] "Zm00001eb091010" "Zm00001eb091080" "Zm00001eb091230" "Zm00001eb091430"
 [157] "Zm00001eb091890" "Zm00001eb091940" "Zm00001eb093110" "Zm00001eb093140"
 [161] "Zm00001eb093200" "Zm00001eb093280" "Zm00001eb093290" "Zm00001eb093300"
 [165] "Zm00001eb093320" "Zm00001eb093380" "Zm00001eb093450" "Zm00001eb093710"
 [169] "Zm00001eb093890" "Zm00001eb093980" "Zm00001eb094220" "Zm00001eb094310"
 [173] "Zm00001eb094350" "Zm00001eb094400" "Zm00001eb094420" "Zm00001eb094550"
 [177] "Zm00001eb094560" "Zm00001eb094840" "Zm00001eb094860" "Zm00001eb094890"
 [181] "Zm00001eb094910" "Zm00001eb095100" "Zm00001eb095120" "Zm00001eb095240"
 [185] "Zm00001eb095250" "Zm00001eb095370" "Zm00001eb095560" "Zm00001eb095660"
 [189] "Zm00001eb095750" "Zm00001eb095960" "Zm00001eb096590" "Zm00001eb096820"
 [193] 

In [14]:
head(zm_cell_type_FGSEA)

pathway,pval,padj,ES,NES,nMoreExtreme,size,leadingEdge,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>,<chr>
cell_development,0.001194030,0.1842786,-0.5040418,-2.259442,1,13,"Zm00001eb294970, Zm00001eb020870, Zm00001eb074460, Zm00001eb010460, Zm00001eb189400, Zm00001eb268360, Zm00001eb104820, Zm00001eb001290, Zm00001eb160490, Zm00001eb026330, Zm00001eb297830",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
cellular_component_morphogenesis,0.007119387,0.3325802,-0.4629126,-2.002639,12,12,"Zm00001eb294970, Zm00001eb010460, Zm00001eb189400, Zm00001eb268360, Zm00001eb104820, Zm00001eb001290, Zm00001eb160490, Zm00001eb026330, Zm00001eb297830, Zm00001eb366340, Zm00001eb153760",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
cell_morphogenesis,0.007376185,0.3325802,-0.4761055,-1.962252,13,11,"Zm00001eb294970, Zm00001eb010460, Zm00001eb189400, Zm00001eb268360, Zm00001eb104820, Zm00001eb001290, Zm00001eb160490, Zm00001eb026330, Zm00001eb297830",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
epidermal_cell_differentiation,0.016949153,0.3567026,-0.4595523,-1.790620,33,10,"Zm00001eb020870, Zm00001eb010460, Zm00001eb268360, Zm00001eb104820, Zm00001eb001290, Zm00001eb160490, Zm00001eb026330, Zm00001eb297830",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
cell_cycle_process,0.037787514,0.4529645,-0.3837391,-1.660121,68,12,"Zm00001eb203340, Zm00001eb190840, Zm00001eb294970, Zm00001eb128470, Zm00001eb373160, Zm00001eb104820, Zm00001eb257820, Zm00001eb129850",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
cell_differentiation,0.037564767,0.4529645,-0.2476759,-1.617835,28,27,"Zm00001eb408950, Zm00001eb294970, Zm00001eb172240, Zm00001eb152840, Zm00001eb020870, Zm00001eb074460, Zm00001eb253650, Zm00001eb010460, Zm00001eb189400, Zm00001eb268360, Zm00001eb293080, Zm00001eb104820, Zm00001eb257820, Zm00001eb001290, Zm00001eb160490, Zm00001eb026330, Zm00001eb297830, Zm00001eb336960, Zm00001eb279580, Zm00001eb274840, Zm00001eb343590",Zm.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv


In [15]:
fitlered_save <- zm_cell_type_FGSEA  %>% 
    group_by(celltype)  %>% 
    mutate_at(c('pval'), as.numeric)  %>% 
    dplyr::filter(NES > 0 & pval < .05) %>% 
    dplyr::arrange(pval, descending = TRUE)  %>% 
    dplyr::arrange(celltype)  %>% 
    mutate(celltype = str_replace(celltype, "Zm.de_novo.", ""))  %>% 
    mutate(celltype = str_replace(celltype, ".upregulated_genes.deseq2_output.tsv", ""))  %>% 
    dplyr::select(-leadingEdge)

write_delim(fitlered_save, "/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/supplament/Zm.FGSEA_results.tsv", delim = "\t")

In [20]:
head(fitlered_save)

pathway,pval,padj,ES,NES,nMoreExtreme,size,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
ion_transmembrane_transport,0.0004218964,0.1352220,0.4627763,2.068908,3,35,bundle_sheath
pigment_metabolic_process,0.0005841121,0.1352220,0.6101027,2.191775,4,16,bundle_sheath
pigment_biosynthetic_process,0.0023795360,0.2754313,0.5836389,2.010667,19,14,bundle_sheath
transmembrane_transport,0.0034832497,0.3225489,0.3754350,1.829859,33,53,bundle_sheath
small_molecule_catabolic_process,0.0074178735,0.3325802,0.5283518,1.861746,62,15,bundle_sheath
alpha_amino_acid_metabolic_process,0.0074273412,0.3325802,0.4206696,1.789722,68,29,bundle_sheath
